# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7b38037460>
├── 'a' --> tensor([[-1.5873, -0.6847, -0.4494],
│                   [-1.5296,  3.3561, -0.5296]])
└── 'x' --> <FastTreeValue 0x7f7b38088b20>
    └── 'c' --> tensor([[ 0.7738, -0.2115, -0.0797, -0.3335],
                        [-0.6632,  1.0757, -0.9225, -0.4106],
                        [-2.1503,  0.1796, -1.0458, -1.2582]])

In [4]:
t.a

tensor([[-1.5873, -0.6847, -0.4494],
        [-1.5296,  3.3561, -0.5296]])

In [5]:
%timeit t.a

57.6 ns ± 0.0588 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7b38037460>
├── 'a' --> tensor([[ 0.0054, -1.1224,  0.7509],
│                   [-0.2868, -0.8945, -2.0345]])
└── 'x' --> <FastTreeValue 0x7f7b38088b20>
    └── 'c' --> tensor([[ 0.7738, -0.2115, -0.0797, -0.3335],
                        [-0.6632,  1.0757, -0.9225, -0.4106],
                        [-2.1503,  0.1796, -1.0458, -1.2582]])

In [7]:
%timeit t.a = new_value

70.3 ns ± 0.0285 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5873, -0.6847, -0.4494],
               [-1.5296,  3.3561, -0.5296]]),
    x: Batch(
           c: tensor([[ 0.7738, -0.2115, -0.0797, -0.3335],
                      [-0.6632,  1.0757, -0.9225, -0.4106],
                      [-2.1503,  0.1796, -1.0458, -1.2582]]),
       ),
)

In [10]:
b.a

tensor([[-1.5873, -0.6847, -0.4494],
        [-1.5296,  3.3561, -0.5296]])

In [11]:
%timeit b.a

52.4 ns ± 0.0256 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.8777,  0.4581, -1.6927],
               [-0.0432, -0.4496, -1.4646]]),
    x: Batch(
           c: tensor([[ 0.7738, -0.2115, -0.0797, -0.3335],
                      [-0.6632,  1.0757, -0.9225, -0.4106],
                      [-2.1503,  0.1796, -1.0458, -1.2582]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.223 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

844 ns ± 2.84 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 6.93 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 2.89 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 613 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7a7e227790>
├── 'a' --> tensor([[[-1.5873, -0.6847, -0.4494],
│                    [-1.5296,  3.3561, -0.5296]],
│           
│                   [[-1.5873, -0.6847, -0.4494],
│                    [-1.5296,  3.3561, -0.5296]],
│           
│                   [[-1.5873, -0.6847, -0.4494],
│                    [-1.5296,  3.3561, -0.5296]],
│           
│                   [[-1.5873, -0.6847, -0.4494],
│                    [-1.5296,  3.3561, -0.5296]],
│           
│                   [[-1.5873, -0.6847, -0.4494],
│                    [-1.5296,  3.3561, -0.5296]],
│           
│                   [[-1.5873, -0.6847, -0.4494],
│                    [-1.5296,  3.3561, -0.5296]],
│           
│                   [[-1.5873, -0.6847, -0.4494],
│                    [-1.5296,  3.3561, -0.5296]],
│           
│                   [[-1.5873, -0.6847, -0.4494],
│                    [-1.5296,  3.3561, -0.5296]]])
└── 'x' --> <FastTreeValue 0x7f7a7e2273d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 35.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7a84e88610>
├── 'a' --> tensor([[-1.5873, -0.6847, -0.4494],
│                   [-1.5296,  3.3561, -0.5296],
│                   [-1.5873, -0.6847, -0.4494],
│                   [-1.5296,  3.3561, -0.5296],
│                   [-1.5873, -0.6847, -0.4494],
│                   [-1.5296,  3.3561, -0.5296],
│                   [-1.5873, -0.6847, -0.4494],
│                   [-1.5296,  3.3561, -0.5296],
│                   [-1.5873, -0.6847, -0.4494],
│                   [-1.5296,  3.3561, -0.5296],
│                   [-1.5873, -0.6847, -0.4494],
│                   [-1.5296,  3.3561, -0.5296],
│                   [-1.5873, -0.6847, -0.4494],
│                   [-1.5296,  3.3561, -0.5296],
│                   [-1.5873, -0.6847, -0.4494],
│                   [-1.5296,  3.3561, -0.5296]])
└── 'x' --> <FastTreeValue 0x7f7b0c4d1f10>
    └── 'c' --> tensor([[ 0.7738, -0.2115, -0.0797, -0.3335],
                        [-0.6632,  1.0757, -0.9225, -0.4106],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 38 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.8 µs ± 59.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7738, -0.2115, -0.0797, -0.3335],
                       [-0.6632,  1.0757, -0.9225, -0.4106],
                       [-2.1503,  0.1796, -1.0458, -1.2582]],
              
                      [[ 0.7738, -0.2115, -0.0797, -0.3335],
                       [-0.6632,  1.0757, -0.9225, -0.4106],
                       [-2.1503,  0.1796, -1.0458, -1.2582]],
              
                      [[ 0.7738, -0.2115, -0.0797, -0.3335],
                       [-0.6632,  1.0757, -0.9225, -0.4106],
                       [-2.1503,  0.1796, -1.0458, -1.2582]],
              
                      [[ 0.7738, -0.2115, -0.0797, -0.3335],
                       [-0.6632,  1.0757, -0.9225, -0.4106],
                       [-2.1503,  0.1796, -1.0458, -1.2582]],
              
                      [[ 0.7738, -0.2115, -0.0797, -0.3335],
                       [-0.6632,  1.0757, -0.9225, -0.4106],
                       [-2.1503,  0.1796, -1.0458, -1.2582]],

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7738, -0.2115, -0.0797, -0.3335],
                      [-0.6632,  1.0757, -0.9225, -0.4106],
                      [-2.1503,  0.1796, -1.0458, -1.2582],
                      [ 0.7738, -0.2115, -0.0797, -0.3335],
                      [-0.6632,  1.0757, -0.9225, -0.4106],
                      [-2.1503,  0.1796, -1.0458, -1.2582],
                      [ 0.7738, -0.2115, -0.0797, -0.3335],
                      [-0.6632,  1.0757, -0.9225, -0.4106],
                      [-2.1503,  0.1796, -1.0458, -1.2582],
                      [ 0.7738, -0.2115, -0.0797, -0.3335],
                      [-0.6632,  1.0757, -0.9225, -0.4106],
                      [-2.1503,  0.1796, -1.0458, -1.2582],
                      [ 0.7738, -0.2115, -0.0797, -0.3335],
                      [-0.6632,  1.0757, -0.9225, -0.4106],
                      [-2.1503,  0.1796, -1.0458, -1.2582],
                      [ 0.7738, -0.2115, -0.0797, -0.3335],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

313 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
